In [ ]:
def basic_block(x, num_features=128, weight_decay=0):
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [ ]:
def se_fn(x,num_features=128,amplifying_ratio=16):
#     num_features = x.shape[-1].value
    shortcut=x
    num_features=128
    x = GlobalAvgPool1D()(x)
    x = Reshape((1, num_features))(x)
    x = Dense(num_features * amplifying_ratio, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(num_features, activation='sigmoid', kernel_initializer='glorot_uniform')(x)
    x = Multiply()([x,shortcut])
    return x

In [ ]:
def rese_block(x, num_features=128, weight_decay=0, amplifying_ratio=16):
#     if num_features != x.shape[-1].value:
#         shortcut = Conv1D(num_features, kernel_size=1, padding='same', use_bias=True,kernel_regularizer=l2(weight_decay), kernel_initializer='glorot_uniform')(x)
#         shortcut = BatchNormalization()(shortcut)
#     else:
#         shortcut = x
    num_features=128
    x = basic_block(x,num_features,weight_decay)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(num_features, kernel_size=3, padding='same', use_bias=True,kernel_regularizer=l2(weight_decay), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    shortcut=x
    if amplifying_ratio > 0:
        x = se_fn(x, amplifying_ratio)
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3)(x)
    return x

In [ ]:
layer1=Input(shape=(16000,8))
layer2=basic_block1(layer1)
layer3=MaxPool1D(pool_size=3,strides=None)(layer2)
layer4=rese_block(layer3)
layer5=rese_block(layer4)
layer6=rese_block(layer5)
layer7=rese_block(layer6)
layer8=rese_block(layer7)
layer9=GlobalMaxPool1D()(layer8)
final=Dense(units=256, activation='relu')(layer9)
layer10=BatchNormalization()(final)
layer11=Activation(activation)(layer10)    
layer12=Dropout(dropout_rate)(layer11)
out_class=Dense(units=8, activation='softmax')(layer12)
layer14=BatchNormalization()(final)
layer15=Activation(activation)(layer14)    
layer16=Dropout(dropout_rate)(layer15)
out_reg=Dense(units=2, activation='linear')(layer16)
model=keras.Model(inputs=layer1,outputs=[out_class,out_reg])
model.summary()